# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [2]:
from etl_helper import *

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
#     print(file_path_list)

/home/rvdubey/workspace/data_eng_nanodegree/data_modeling_cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
session.execute("""
CREATE KEYSPACE IF NOT EXISTS sparkify
WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}""")

#### Set Keyspace

In [8]:
session.set_keyspace('sparkify')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [9]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

create_table_query = """
CREATE TABLE IF NOT EXISTS
sessionized_music_table
    (
    sessionId int, 
    itemInSession int,
    artist text,
    song text,
    length float,
    PRIMARY KEY (sessionId, itemInSession))"""
session.execute(create_table_query)

insert_data_in_table(session, 'sessionized_music_table', 
                     ('sessionId', 'itemInSession', 'artist', 'song', 'length')
                    )

result = session.execute("""
SELECT artist, song, length FROM sessionized_music_table where sessionId=338 AND itemInSession=4""")

print (f"""\nResults for:
Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4\n""")
for r in result:
    print(r)                 

Inserting data into sessionized_music_table...
Finished inserting data into sessionized_music_table

Results for:
Query 1:  Give me the artist, song title and song's length in the music app history that was heard during ## sessionId = 338, and itemInSession = 4

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


In [10]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
create_table_query = """
CREATE TABLE IF NOT EXISTS
user_sessionized_music_table
    (
    sessionId int,
    userId int,
    itemInSession int,
    artist text,
    song text,
    firstName text,
    lastName text,
    PRIMARY KEY ((sessionId, userId), itemInSession)
    )
    WITH CLUSTERING ORDER BY (itemInSession DESC);"""
session.execute(create_table_query)

insert_data_in_table(session, 'user_sessionized_music_table', 
                     ('sessionId', 'userId', 'itemInSession', 'artist', 'song', 'firstName', 'lastName')
                    )

print (f"""\nResults for:
Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182\n""")
result = session.execute("""
SELECT artist, song, firstName, lastName, itemInSession FROM user_sessionized_music_table where sessionId=182 AND userId=10""")
for r in result:
    print(r)

Inserting data into user_sessionized_music_table...
Finished inserting data into user_sessionized_music_table

Results for:
Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)## for userid = 10, sessionid = 182

Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz', iteminsession=3)
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz', iteminsession=2)
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz', iteminsession=1)
Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz', iteminsession=0)


In [11]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
create_table_query = """
CREATE TABLE IF NOT EXISTS
songs_music_table
    (
    firstName text,
    lastName text,
    song text,
    PRIMARY KEY (song)
    );"""
session.execute(create_table_query)

insert_data_in_table(session, 'songs_music_table', 
                     ('firstName', 'lastName', 'song')
                    )
print (f"""\nResults for:
Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'\n""")
result = session.execute("""
SELECT  firstName, lastName  FROM songs_music_table where song='All Hands Against His Own'""")
for r in result:
    print(r)

Inserting data into songs_music_table...
Finished inserting data into songs_music_table

Results for:
Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Row(firstname='Tegan', lastname='Levine')


### Drop the tables before closing out the sessions

In [12]:
table_list = session.execute("""SELECT table_name FROM system_schema.tables WHERE keyspace_name = 'sparkify';""")
for t in table_list:
    print(t.table_name)
    session.execute(f"DROP TABLE IF EXISTS {t.table_name}")

sessionized_music_table
songs_music_table
user_sessionized_music_table


### Close the session and cluster connection¶

In [13]:
session.shutdown()
cluster.shutdown()